In [10]:
import os, math, json, time, random, pandas as pd
from pprint import pprint
import concurrent.futures

In [9]:
def machine(data):
#     print(data, type(data))
    # INITIALIZATION
#     spiders = []
#     MAX_THREAD = 2
#     # CHUNK SPIDER
#     step_n = math.ceil(len(data) / MAX_THREAD) # Return the ceiling of step_n, the smallest integer greater than or equal to step_n
#     spider_data = [data[i:i + step_n]
#                    for i in range(0, len(data), step_n)]
#     # SPIDER CRAWLER
# #     print("Total thread spider(s): {}".format(len(spider_data)))
#     # print("Divisible_n:", divisible_n)
#     for spider in spider_data:
#         spiders.append(len(spider))
    return [len(data)]

In [8]:
def spider(data):
    # INITIALIZATION
    spiders = []
    MAX_THREAD = 2
    # CHUNK SPIDER
    step_n = math.ceil(len(data) / MAX_THREAD) # Return the ceiling of step_n, the smallest integer greater than or equal to step_n
    spider_data = [data[i:i + step_n]
                   for i in range(0, len(data), step_n)]
    # SPIDER CRAWLER
#     print("Total thread spider(s): {}".format(len(spider_data)))
    # print("Divisible_n:", divisible_n)
    for spider in spider_data:
        spiders.append(len(spider))
    return spiders

In [7]:
def main(system_data, WORKERS):
    # Computation for divisible number for chunking URL(s)
    divisible_n = math.ceil(len(system_data) / WORKERS)
    data = [system_data[i:i + divisible_n]
            for i in range(0, len(system_data), divisible_n)]  # Chunking URL(s)

    # Instansiating MAX WORKERS based on the lower number between length of data and defined WORKERS
    MAX_WORKERS = min(len(data), WORKERS)

    # LOGGING ---------------------------
    print("Total Data: {}".format(len(system_data)))
    print("Total Worker(s): {}".format(MAX_WORKERS))
    print("Chunked Data for multiprocessing: ",
          list(map(lambda d: len(d), data)))
    print("----"*25)
    # MULTI PROCESSING OF SPIDER FUNCTION
    with concurrent.futures.ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
        future = [executor.submit(spider, obj) for obj in data]

    # Getting the return value of multi spiders
    spiders = [obj.result() for obj in future]
    return len(system_data), MAX_WORKERS, spiders

In [6]:
def main_machine(system_data, WORKERS):
    # Computation for divisible number for chunking data
    divisible_n = math.ceil(len(system_data) / WORKERS)
    data = [system_data[i:i + divisible_n]
            for i in range(0, len(system_data), divisible_n)]  # Chunking Data

    # Instansiating MAX WORKERS based on the lower number between length of data and defined WORKERS
    MAX_MACHINE = min(len(data), WORKERS)

    # LOGGING ---------------------------
    print("Total Data: {}".format(len(system_data)))
    print("Total Available Machine(s): {}".format(MAX_MACHINE))
    print("Chunked Data for Distribution: ",
          list(map(lambda d: len(d), data)))
    print("----"*25)
    # MULTI PROCESSING OF SPIDER FUNCTION
    with concurrent.futures.ProcessPoolExecutor(max_workers=MAX_MACHINE) as executor:
        future = [executor.submit(machine, obj) for obj in data]

    # Getting the return value of multi spiders
    spiders = [obj.result() for obj in future]
    return len(system_data), MAX_MACHINE, spiders

In [5]:
def get_headers(total_workers, column_name="Worker"):
    headers = []
    for i in range(1, total_workers + 1):
        headers.append("{} {}".format(column_name, i))
    return headers

In [4]:
def get_rows(spiders, row_name="Thread"):
    rows = []
    for i in range(1, len(spiders[0]) + 1):
        rows.append("{} {}".format(row_name, i))
    return rows

In [3]:
def consumers_distribution_details(data=500, machine=5):
    
    DATA = list(range(data))
    WORKERS = machine
    total_data, total_workers, spiders = main_machine(DATA, WORKERS)
    headers = get_headers(total_workers, column_name="Machine")
    fields = get_rows(spiders, row_name="Data")
    data_set = pd.DataFrame(spiders, index = headers, columns = fields)
    print(data_set.T)

In [2]:
def machine_details(data=500, workers=14):
    
    DATA = list(range(data))
    WORKERS = workers
    total_data, total_workers, spiders = main(DATA, WORKERS)
    headers = get_headers(total_workers)
    fields = get_rows(spiders)
    data_set = pd.DataFrame(spiders, index = headers, columns = fields)
    print(data_set.T)

In [11]:
consumers_distribution_details(data=5000, machine=3)

Total Data: 5000
Total Available Machine(s): 3
Chunked Data for Distribution:  [1667, 1667, 1666]
----------------------------------------------------------------------------------------------------
        Machine 1  Machine 2  Machine 3
Data 1       1667       1667       1666


In [14]:
machine_details(data=1667, workers =14)

Total Data: 1667
Total Worker(s): 14
Chunked Data for multiprocessing:  [120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 107]
----------------------------------------------------------------------------------------------------
          Worker 1  Worker 2  Worker 3  Worker 4  Worker 5  Worker 6  \
Thread 1        60        60        60        60        60        60   
Thread 2        60        60        60        60        60        60   

          Worker 7  Worker 8  Worker 9  Worker 10  Worker 11  Worker 12  \
Thread 1        60        60        60         60         60         60   
Thread 2        60        60        60         60         60         60   

          Worker 13  Worker 14  
Thread 1         60         54  
Thread 2         60         53  
